# 🔗 Introduction to LangChain Expression Language (LCEL)

## What is LCEL?

**LangChain Expression Language (LCEL)** is a declarative way to compose chains in LangChain. It provides a simple, intuitive syntax using the **pipe operator (`|`)** to connect different components together.

### Key Benefits of LCEL:
- **🚀 Simplified Syntax**: Chain components using the intuitive `|` operator
- **⚡ First-class Streaming Support**: Get tokens as soon as they're available from the LLM
- **🔄 Async Support**: Same chain works for both sync and async operations
- **🔧 Optimized Parallel Execution**: Automatically runs independent steps in parallel
- **📝 Transparent Tracing**: Easy debugging with LangSmith integration

### How LCEL Works:
```
Input → Component1 | Component2 | Component3 → Output
```

Each component in the chain:
1. Receives input from the previous component
2. Processes it
3. Passes the output to the next component

---

## Learning Objectives:
By the end of this notebook, you will:
1. Understand the basic LCEL syntax
2. Create a simple LLM chain using LCEL
3. Execute the chain with custom inputs
4. See a practical example with SQL query generation

In [ ]:
# ============================================================
# 📦 STEP 1: Install Required Libraries (Run once if needed)
# ============================================================
# 
# langchain: The core LangChain library for building LLM applications
# langchain-openai: Integration package for OpenAI models with LangChain
#
# Uncomment the lines below if you haven't installed these packages:

# !pip install -qq langchain==0.3.11
# !pip install -qq langchain-openai==0.2.12

In [ ]:
# langchain_community: Contains community-contributed integrations
# (Required for SQLDatabase utility used later in this notebook)

# !pip install -qq langchain_community

---

## 🔑 Step 2: Setup OpenAI API Credentials

Before using OpenAI models, you need to configure your API key. We use the `python-dotenv` library to securely load environment variables from a `.env` file.

**Important**: Create a `.env` file in your project root with:
```
OPENAI_API_KEY=your-api-key-here
```

> ⚠️ **Security Tip**: Never hardcode API keys in your notebooks or commit them to version control!

In [ ]:
import os
from dotenv import load_dotenv

# load_dotenv() reads the .env file and loads all key-value pairs
# as environment variables. Returns True if successful.
load_dotenv()

---

## 🤖 Step 3: Initialize the LLM (Chat Model)

LangChain provides the `ChatOpenAI` class to interact with OpenAI's chat models. This is a **Runnable** component - the building block of LCEL chains.

### Key Parameters:
- `model_name`: The specific model to use (e.g., "gpt-4o-mini", "gpt-4", "gpt-3.5-turbo")
- `temperature`: Controls randomness (0 = deterministic, 1 = creative)

In [ ]:
# Import the ChatOpenAI class from langchain_openai package
from langchain_openai import ChatOpenAI

# Initialize the Chat Model
# - model_name: "gpt-4o-mini" is a cost-effective, fast model good for learning
# - temperature=0: Makes responses deterministic and focused (no randomness)
chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# The 'chatgpt' object is now a Runnable that can be used in LCEL chains
print(f"Model initialized: {chatgpt.model_name}")

---

## ⛓️ Step 4: Create an LCEL Chain

Now comes the exciting part - building our first LCEL chain!

### The Pipe Operator (`|`)
In LCEL, the `|` operator connects components together:
```python
chain = component1 | component2 | component3
```

### Our Chain Structure:
```
User Input → PromptTemplate → ChatModel → Output
     ↓              ↓              ↓
  {"topic":    Formats the     Generates
   "AI"}       prompt with      response
               the topic
```

### What is a PromptTemplate?
A `PromptTemplate` is a reusable template that:
1. Accepts variables (like `{topic}`)
2. Formats them into a proper prompt for the LLM

In [ ]:
# Import ChatPromptTemplate - used to create structured prompts
from langchain_core.prompts import ChatPromptTemplate

# ============================================================
# STEP 4a: Define the Prompt Template
# ============================================================
# The {topic} is a placeholder that will be replaced with actual values
# when we invoke the chain

prompt_txt = "Explain {topic} in 1 line"

# Create a ChatPromptTemplate from the string template
# This automatically converts our string into a proper chat message format
prompt_template = ChatPromptTemplate.from_template(prompt_txt)

# Let's see what the template looks like
print("📋 Prompt Template Variables:", prompt_template.input_variables)

# ============================================================
# STEP 4b: Build the LCEL Chain using the Pipe Operator
# ============================================================
# The "|" operator chains components together:
#   - Input flows from LEFT to RIGHT
#   - Each component processes and passes data to the next

# Method 1: Multi-line format (better readability for complex chains)
llmchain = (
    prompt_template  # First: Format the prompt with input variables
    |                # Pipe operator: Pass output to next component
    chatgpt          # Second: Send formatted prompt to LLM
)

# Method 2: Single-line format (equivalent, more compact)
# llmchain = prompt_template | chatgpt

print("✅ Chain created successfully!")

---

## 🚀 Step 5: Execute the Chain

Now let's run our chain! Every LCEL chain has these key methods:

| Method | Description |
|--------|-------------|
| `.invoke()` | Run the chain synchronously (waits for complete response) |
| `.stream()` | Stream tokens as they're generated |
| `.batch()` | Process multiple inputs in parallel |
| `.ainvoke()` | Async version of invoke |

### Using `.invoke()`
The `invoke()` method:
1. Takes a dictionary with input variables
2. Passes them through each component
3. Returns the final output

In [ ]:
# ============================================================
# Execute the chain with .invoke()
# ============================================================
# Pass a dictionary with the required variable(s)
# The key 'topic' matches the {topic} placeholder in our template

response = llmchain.invoke({'topic': 'Generative AI'})

# The response is an AIMessage object from LangChain
# - response.content: The actual text response from the LLM
# - response.response_metadata: Additional info (tokens used, model, etc.)

print("🤖 LLM Response:")
print("-" * 50)
print(response.content)
print("-" * 50)
print(f"\n📊 Response Type: {type(response).__name__}")

---

## 🎯 Bonus: Real-World Example - SQL Query Generation

Let's see LCEL in action with a practical use case: **generating SQL queries from natural language questions**.

LangChain provides a pre-built chain `create_sql_query_chain` that:
1. Connects to your database
2. Understands the schema
3. Converts natural language to SQL

This demonstrates how LCEL enables complex, production-ready chains!


In [ ]:
# ============================================================
# SQL Query Generation Example using LCEL
# ============================================================

from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

# Step 1: Connect to the database
# SQLDatabase.from_uri() creates a connection to your database
# Here we're using the Chinook sample database (a music store DB)
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

# Step 2: Initialize a new LLM instance for SQL generation
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Step 3: Create the SQL query chain
# This pre-built chain uses LCEL internally and:
#   - Analyzes the database schema
#   - Understands table relationships
#   - Generates appropriate SQL queries
sql_chain = create_sql_query_chain(llm, db)

# Step 4: Invoke the chain with a natural language question
response = sql_chain.invoke({"question": "How many employees are there"})

print("💬 Question: How many employees are there?")
print("📝 Generated SQL Query:")
print(response)

# ============================================================
# 💡 Key Insight:
# The LLM understood our natural language question and:
# 1. Identified the relevant table (Employees)
# 2. Generated the correct SQL syntax
# 3. Used COUNT() aggregate function appropriately
# ============================================================

In [ ]:
# ============================================================
# 💡 Key Insight:
# ============================================================
# The LLM understood our natural language question and:
# 1. Identified the relevant table (Employees)
# 2. Generated the correct SQL syntax
# 3. Used COUNT() aggregate function appropriately
#
# Try asking different questions like:
# - "What are the names of all artists?"
# - "Show me the top 5 most expensive tracks"
# - "How many albums does each artist have?"
# ============================================================

# Execute the generated query on the database to verify it works
result = db.run(response)
print(f"✅ Query Result: {result}")


## 📚 Summary

Congratulations! You've learned the fundamentals of LCEL. Here's what we covered:

### Key Takeaways:

| Concept | Description |
|---------|-------------|
| **LCEL** | LangChain Expression Language - a declarative way to compose chains |
| **Pipe Operator (`\|`)** | Connects components, data flows left to right |
| **Runnable** | Any component that can be part of an LCEL chain |
| **PromptTemplate** | Reusable template with variable placeholders |
| **invoke()** | Method to execute a chain with input data |

### The LCEL Pattern:
```python
chain = component1 | component2 | component3
result = chain.invoke({"input_key": "input_value"})
```

---

## 🏋️ Practice Exercises

Try these exercises to reinforce your learning:

1. **Modify the prompt**: Change the prompt to "Explain {topic} to a 5-year-old" and test with different topics

2. **Add more variables**: Create a prompt with multiple variables like "Explain {topic} in {language} language"

3. **Try different models**: Replace `gpt-4o-mini` with `gpt-3.5-turbo` and compare responses

4. **Experiment with temperature**: Try `temperature=0.7` or `temperature=1.0` and observe the differences

---

## 🔗 Next Steps

Continue your LCEL journey by exploring:
- **Runnables**: Learn about RunnablePassthrough, RunnableLambda, and more
- **Output Parsers**: Structure LLM outputs into Python objects
- **Chains**: Build more complex multi-step chains
- **Streaming**: Get real-time token-by-token responses
